# Lab-10-5 Advanced CNN(VGG)

## 지난시간까지
- ImageFolder

## 오늘은
- VGG Network 만들기

## 이론적인 설명은 어디서?
- [모두를 위한 딥러닝 시즌 1](https://www.youtube.com/watchv=KbNbWTnlYXs&list=PLlMkM4tgfjnLSOjrEJN31gZATbcj_MpUm&index=38)
- [Pytorch Lecture 11: Advanced CNN](https://www.youtube.com/watch?v=hqYfqNAQIjE)


In [1]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

In [2]:
__all__ = [
    "VGG", "vgg11", "vgg11_bn", "vgg13", "vgg13_bn", "vgg16", "vgg16_bn",
    "vgg19_bn", "vgg19",
]

model_urls = {
    "vgg11" : "https://download.pytorch.org/models/vgg11_bbd30ac9.pth",
    "vgg13" : "https://download.pytorch.org/models/vgg13_c768596a.pth",
    "vgg16" : "https://download.pytorch.org/models/vgg16_397923af.pth",
    "vgg19" : "https://download.pytorch.org/models/vgg19_dcbb9e9d.pth",
    "vgg11_bn" : "https://download.pytorch.org/models/vgg11_bn-6002323d.pth",
    "vgg13_bn" : "https://download.pytorch.org/models/vgg13_bn-abd245e5.pth",
    "vgg16_bn" : "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth",
    "vgg19_bn" : "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth"
}

In [19]:
class VGG(nn.Module):
    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        
        self.features = features #convolution
        
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),   # FC Layer 1개째
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),          # FC Layer 2개째
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),   # FC Layer 3개째
        )#FC layer
        
        if init_weights:     # weight initialize하는 부분
            self._initialize_weights()
        
    def forward(self, x):
        x = self.features(x) #Convolution
        x = self.avgpool(x) # avgpool
        x = x.view(x.size(0), -1) # 일렬로 펼치기
        x = self.classifier(x) #FC layer
        return x

    def _initialize_weights(self):
        for m in self.modules():   # feature의 레이어 값을 하나씩 리턴. m에게
            if isinstance(m, nn.Conv2d):
                # m이 처음에는 컨볼루션일거고, 그때는 이걸로 normalize해서 초기화해라
                # kaiming he라는 박사님이 제안한, activation func에 따른 초기화 방법
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [5]:
def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == "M":
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)
    
    
'''
"A" : [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
# M은 maxpooling

->

conv2d = nn.Conv2d(3, 64, kernel_size=3, padding=1), #1
nn.ReLU(inplace=True)

nn.MaxPool2d(kernel_size=2, stride=2)

conv2d = nn.Conv2d(64, 128, kernel_size=3, padding=1) #2
nn.ReLU(inplace=True)

nn.MaxPool2d(kernel_size=2, stride=2)

conv2d = nn.Conv2d(128, 256, kernel_size=3, padding=1) #3
nn.ReLU(inplace=True)

conv2d = nn.Conv2d(256, 256, kernel_size=3, padding=1) #4
nn.ReLU(inplace=True)

nn.MaxPool2d(kernel_size=2, stride=2)

conv2d = nn.Conv2d(256, 512, kernel_size=3, padding=1) #5
nn.ReLU(inplace=True)

conv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1) #6
nn.ReLU(inplace=True)

nn.MaxPool2d(kernel_size=2, stride=2)

conv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1) #7
nn.ReLU(inplace=True)

conv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1) #8
nn.ReLU(inplace=True)

nn.MaxPool2d(kernel_size=2, stride=2)


: convolution layer 8개

'''

'\n"A" : [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],\n\n->\n\nconv2d = nn.Conv2d(3, 64, kernel_size=3, padding=1), #1\nnn.ReLU(inplace=True)\n\nnn.MaxPool2d(kernel_size=2, stride=2)\n\nconv2d = nn.Conv2d(64, 128, kernel_size=3, padding=1) #2\nnn.ReLU(inplace=True)\n\nnn.MaxPool2d(kernel_size=2, stride=2)\n\nconv2d = nn.Conv2d(128, 256, kernel_size=3, padding=1) #3\nnn.ReLU(inplace=True)\n\nconv2d = nn.Conv2d(256, 256, kernel_size=3, padding=1) #4\nnn.ReLU(inplace=True)\n\nnn.MaxPool2d(kernel_size=2, stride=2)\n\nconv2d = nn.Conv2d(256, 512, kernel_size=3, padding=1) #5\nnn.ReLU(inplace=True)\n\nconv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1) #6\nnn.ReLU(inplace=True)\n\nnn.MaxPool2d(kernel_size=2, stride=2)\n\nconv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1) #7\nnn.ReLU(inplace=True)\n\nconv2d = nn.Conv2d(512, 512, kernel_size=3, padding=1) #8\nnn.ReLU(inplace=True)\n\nnn.MaxPool2d(kernel_size=2, stride=2)\n\n\n: convolution layer 8개\n\n'

In [13]:
cfg = {
    "A" : [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],   # conv 레이어 8개, fc layer 3개 -> vgg11 
    "B" : [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"], # 10 + 3 = vgg 13
    "D" : [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],  # 13 + 3 = vgg16
    "E" : [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"],  # 16 + 3 = vgg19
    "custom" : [64, 64, 64, "M", 128, 128, 128, "M", 256, 256, 256, "M"] # 이렇게 웒는 레이어도 넣을 수 있다.
}

In [12]:
conv = make_layers(cfg['A'])
conv

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12): ReLU(inplace=True)
  (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (14): ReLU(inplace=True)
  (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (16): Conv2d(512, 512, kernel_size=(3, 3), stride=

In [15]:
conv = make_layers(cfg["custom"], batch_norm=True)
# 일케 하면 conv 다음에 batch
conv

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU(inplace=True)
  (13): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (14): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats

그냥 vgg 레이어 쌓은, sequential 출력하는 것만 본것.
완성본은 아래

In [ ]:
class VGG(nn.Module):
    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        
        self.features = features #convolution
        
        self.avgpool = nn.AdaptivePool2d((7, 7))
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),   # FC Layer 1개째
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),          # FC Layer 2개째
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),   # FC Layer 3개째
        )#FC layer
        
        if init_weights:     # weight initialize하는 부분
            self._initialize_weights()
        
    def forward(self, x):
        x = self.features(x) #Convolution
        x = self.avgpool(x) # avgpool
        x = x.view(x.size(0), -1) # 일렬로 펼치기
        x = self.classifier(x) #FC layer
        return x

    def _initialize_weights(self):
        for m in self.modules():   # feature의 레이어 값을 하나씩 리턴. m에게
            if isinstance(m, nn.Conv2d):
                # m이 처음에는 컨볼루션일거고, 그때는 이걸로 normalize해서 초기화해라
                # kaiming he라는 박사님이 제안한, activation func에 따른 초기화 방법
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [16]:
conv = make_layers(cfg["custom"], batch_norm=True)

In [21]:
# CIFAR 10 (class 10개)
CNN = VGG(make_layers(cfg["custom"]), num_classes=10, init_weights=True)
CNN

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), paddin